## Generate gauge for provided URL

## Make table showing comments that cause most discussion

Generate a list of strings showing discussed articles

- What makes a article most discussed
    no. of comments
- top 5 stories with most comments
- time: 

- When provided a URL
- Show most discussed comments for a given article
- select top comments of article, and show kids.

In [2]:
import psycopg2
from psycopg2 import connect
from dotenv import load_dotenv
from os import environ
import pandas as pd
import textblob
from textblob import TextBlob
import plotly.graph_objects as go
import numpy as np
import streamlit
import json
from requests import get
import html

from plotly.offline import init_notebook_mode, iplot

#get story from id
    - take in url, match to url in database
    - return story_id

In [3]:
def get_db_connection():
    """Returns a database connection."""
    load_dotenv()
    return psycopg2.connect(
        host=environ["DB_HOST"],
        port=environ["DB_PORT"],
        database=environ["DB_NAME"],
        user=environ["DB_USER"],
        password=environ["DB_PASSWORD"]
    )

In [33]:
# def load_stories_data_from_url(url:str) -> pd.DataFrame:
#     """Loads re with greatest score change over last 24hrs from RDS.
#     Returns them as a Dataframe object."""
#     query =f"""
#         SELECT stories.story_id, 
#                 stories.title, 
#                 stories.author,
#                 stories.story_url,
#                 records.comments, 
#                 records.record_id, 
#                 stories.topic_id,
#                 records.score
#         FROM records
#         LEFT JOIN stories ON records.story_id = stories.story_id
#         WHERE stories.story_url LIKE {url}
#         ;
#         """
#     return pd.read_sql(query, con=get_db_connection())

def get_story_id_from_url(url: str) -> pd.DataFrame:
    """Loads re with greatest score change over last 24hrs from RDS.
    Returns them as a Dataframe object."""
    query = f"""
        SELECT story_id
        FROM stories
        WHERE stories.story_url LIKE %(url)s;
        """
    df = pd.read_sql(query, con=get_db_connection(), params={"url": url})
    return df['story_id']

In [34]:
url = 'https://en.bitcoinsistemi.com/breaking-gary-gensler-says-secs-x-account-hacked-bitcoin-spot-etfs-not-yet-approved/'

df_specific_url = get_story_id_from_url(f"{url}")
print(df_specific_url)

0    38932509
Name: story_id, dtype: int64


/var/folders/2f/000yp3dd3t359b791hdtl7jm0000gp/T/ipykernel_70147/2335271775.py:28: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [ ]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/item/"
def get_comment_ids(story: int) -> list:
    """Returns the id of comments from given story on Hacker News."""
    story_info = get(BASE_URL + f"{story}.json", timeout=30).json()
    comment_ids = story_info.get("kids")
    return comment_ids

In [ ]:
list_kids_num = []
    for comment in comments[0:5]:
        comment_info = get(BASE_URL + f"{comment}.json", timeout=30).json()
        list_kids_num.append({'parent_id': comment, 'num_of_child': len(comment_info.get("kids"))})
        kids = len(comment_info.get("kids"))
        print(f"{comment} has {kids} replies.")

In [53]:
import html
from bs4 import BeautifulSoup
import pprint
import re

CLEANR = re.compile('<.*?>')


def format_html(text_string: str):
    # Unescape HTML entities
    unescaped_text = html.unescape(text_string)
    # use regex to get rid of html tags
    clean_text = re.sub(CLEANR, '', unescaped_text)
    return clean_text

def get_top_5_most_replied_parent_comments(story_id: int):
    # get all its parent comments
    parent_comments = get_comment_ids(story_id)
    
    parent_comments_list = []
    for parent_comment_id in parent_comments:
        # figure out a comment's title
        comment_info = get(BASE_URL + f"{parent_comment_id}.json", timeout=30).json()
        
        # Check if "text" key exists in comment_info
        comment_text = comment_info.get("text")
        comment_title = format_html(comment_text[:90]) if comment_text else "No title available"
        
        # figure out the number of children it has
        number_of_children = 0
        kids_info = comment_info.get("kids")
        if kids_info is not None:
            number_of_children = len(kids_info)

        # list of dicts where each dictionary is: {parent_comment_id : number_of_children}
        parent_comments_list.append({'title': comment_title, 'number_of_children': number_of_children})

    # sort list of dicts by number_of_children 
    sorted_list = sorted(parent_comments_list, key=lambda comment_dict: comment_dict.get('number_of_children', 0), reverse=True)
    return sorted_list[:5]

pprint.pprint(get_top_5_most_replied_parent_comments(38865518))

[{'number_of_children': 8,
  'title': '> efficiently converts optical power to electrical powerDamn, I '
           'thought it was just '},
 {'number_of_children': 5,
  'title': 'So... I have a really interesting anecdote on "Power over fiber"In '
           '~20'},
 {'number_of_children': 4,
  'title': 'From a reply:  “wow. I expected expensive (two digits), but '
           'indeed: these are _very_ expen'},
 {'number_of_children': 4,
  'title': 'How much optical power could be safely carried over such an '
           'optical fiber in theory? What '},
 {'number_of_children': 4,
  'title': "What's the use-case here? That's the biggest optoisolator I've "
           'ever heard o'}]


In [57]:
top_5_comments = get_top_5_most_replied_parent_comments(38865518)

# for parent_comment in top_5_comments_info:
#     print(f"{parent_comment.get('title')} - - - [{parent_comment.get('number_of_children')} replies]")
text_list = [
    f"{parent_comment.get('title')} - - - [{parent_comment.get('number_of_children')} replies]" for parent_comment in top_5_comments
]
text_list
        

['> efficiently converts optical power to electrical powerDamn, I thought it was just  - - - [8 replies]',
 'So... I have a really interesting anecdote on "Power over fiber"In ~20 - - - [5 replies]',
 'From a reply:  “wow. I expected expensive (two digits), but indeed: these are _very_ expen - - - [4 replies]',
 'How much optical power could be safely carried over such an optical fiber in theory? What  - - - [4 replies]',
 "What's the use-case here? That's the biggest optoisolator I've ever heard o - - - [4 replies]"]

"Comment" + score + replies

In [ ]:
def load_last_two_records(input_story_id: str) -> pd.DataFrame:
    """Loads records from the last 2 hours for a chosen story.
    Returns them as a Dataframe object."""
    query = f"""
        SELECT
            records.story_id,
            MAX(records.comments) - MIN(records.comments) AS comments_count_change,
            stories.title,
            records.record_time
            FROM records
            JOIN stories ON records.story_id = stories.story_id
            WHERE records.story_id = {int(input_story_id)} 
            GROUP BY records.story_id, stories.title, records.record_time
            ORDER BY records.record_time
            DESC LIMIT 2
        ;
        """
    return pd.read_sql(query, con=get_db_connection())